In [1]:
import os
import pandas as pd
import glob

In [2]:
path_input=os.path.join(os.getcwd(), '../data/input/dresden/2019/')
path_output_weather = os.path.join(os.getcwd(), '../data/output/weather/')
path_output=os.path.join(os.getcwd(), '../data/output/')

## Read Data

In [3]:
directors = glob.glob(path_input+"*")

all_files = []
for director in directors:
    all_files=all_files+glob.glob(director+"/*.csv")
    
li = []

for filename in all_files:
    df_read = pd.read_csv(filename, index_col=None, header=0)
    li.append(df_read)

df = pd.concat(li, axis=0, ignore_index=True)

## Drop Columns

In [4]:
#Columns including single value
#The following columns only included one value (or NaN values)
#city : ['dresden']
#p_rack_locks : [False]
#b_state : ['ok']
#b_active : [ True]
#b_battery_pack : [nan '{"percentage": 0}']
#p_special_racks : [nan 0.]
#p_free_special_racks : [nan 0.]

#Columns which highly correlate

#The following columns highly correlate - meaning they are complementary booleans or if one is high, the other is too

#p_bike [corr(p_spot)=-0.999501, complementary boolean to p_spot]

df_cleaned = df.drop(["city", 
                      "p_bike", 
                      "p_rack_locks", 
                      "b_active", 
                      "b_state", 
                      "b_battery_pack", 
                      "p_rack_locks", 
                      "p_special_racks", 
                      "p_free_special_racks"], axis=1)
df_cleaned.head()

,p_bike_racks,p_spot,p_booked_bikes,p_place_type,datetime,b_number,trip,p_uid,p_bikes,p_lat,...,p_free_racks,p_address,b_lock_types,p_number,b_pedelec_battery,p_lng,b_boardcomputer,p_maintenance,p_terminal_type,p_bike_types
0,0,True,0,0,2019-01-20 00:00:00,93716,first,264546,3,51.033548,...,0,NaN,analog_code_lock,4346.0,0.0,13.748800,0,False,NaN,"{""undefined"": 2, ""15"": 1}"
1,0,True,0,0,2019-01-20 09:40:00,93716,start,264546,3,51.033548,...,0,NaN,analog_code_lock,4346.0,0.0,13.748800,0,False,NaN,"{""undefined"": 2, ""15"": 1}"
2,0,True,0,0,2019-01-20 09:58:00,93716,end,264595,2,51.058771,...,0,NaN,analog_code_lock,4389.0,0.0,13.769281,0,False,NaN,"{""15"": 2}"
3,0,True,0,0,2019-01-20 17:36:00,93716,start,264595,3,51.058771,...,0,NaN,analog_code_lock,4389.0,0.0,13.769281,0,False,NaN,"{""15"": 3}"
4,0,True,0,0,2019-01-20 19:39:00,93716,end,100894,1,51.067153,...,0,NaN,analog_code_lock,4361.0,0.0,13.776480,0,False,NaN,"{""15"": 1}"


## Create Trips

In [5]:
# No Duplicates
len(df_cleaned)-len(df_cleaned.drop_duplicates(subset=df_cleaned.columns.difference(["p_lat", "p_lng"])))

0

In [6]:
print("Starts:", len(df_cleaned[df_cleaned["trip"]=="start"]))
print("Ends:", len(df_cleaned[df_cleaned["trip"]=="end"]))
print("Difference:", len(df_cleaned[df_cleaned["trip"]=="start"])-len(df_cleaned[df_cleaned["trip"]=="end"]))

Starts: 565996
Ends: 550350
Difference: 15646


In [7]:
# df_2: start and end of trips
df_2 = df_cleaned[(df_cleaned["trip"]=="start") | (df_cleaned["trip"]=="end")]
print("Dropped", len(df_cleaned)-len(df_2), "first and last Values")

Dropped 290120 first and last Values


In [8]:
# df_3: start and end of trips sorted by b_number and datetime
df_3=df_2.sort_values(["b_number", "datetime"])
df_3[["b_number", "datetime", "trip"]].iloc[500010:500030] # two examples for multiple starts

,b_number,datetime,trip
669292,93554,2019-07-05 13:37:00,start
669293,93554,2019-07-05 13:46:00,end
669294,93554,2019-07-05 14:20:00,start
669295,93554,2019-07-05 14:35:00,end
669296,93554,2019-07-05 16:44:00,start
669297,93554,2019-07-05 16:51:00,start
669298,93554,2019-07-05 17:29:00,end
669299,93554,2019-07-05 18:48:00,start
669300,93554,2019-07-05 19:18:00,end
669301,93554,2019-07-05 20:20:00,start


In [9]:
# sr_1: True if after start entry comes end entry else False
sr_1 = (df_3['trip'] != df_3['trip'].shift())
df_3["valid_start"]=sr_1

# sr_2: 
#sr_2 = (df_3['trip'] != df_3['trip'].shift(-1))
#df_3['valid_end'] = sr_2


In [10]:
# df_4: start and end of trips sorted by b_number and datetime only valid starts
df_4=df_3[df_3["valid_start"]==True]
print("Deleted entries", len(df_3)-len(df_4))
print("Correct entries", len(df_4))

Deleted entries 15646
Correct entries 1100700


In [11]:
# Tests if there are values with two ends after each other
test = df_4["trip"] != df_4["trip"].shift(-1)
print("Entries where Bike X ends with an end and bike Y starts with an end")
pd.DataFrame(test)[pd.DataFrame(test)["trip"]==False]

Entries where Bike X ends with an end and bike Y starts with an end


,trip


In [12]:
df_starts = df_4[df_4["trip"]=="start"].reset_index(drop=True)
df_ends = df_4[df_4["trip"]=="end"].reset_index(drop=True)



In [13]:
print(df_starts.head()[["b_number", "datetime", "trip"]], "\n")
print(df_ends.head()[["b_number", "datetime", "trip"]])

   b_number             datetime   trip
0     11225  2019-04-29 06:31:00  start
1     11225  2019-04-29 11:27:00  start
2     11225  2019-04-29 11:58:00  start
3     11225  2019-04-29 12:22:00  start
4     11225  2019-04-29 13:00:00  start 

   b_number             datetime trip
0     11225  2019-04-29 06:35:00  end
1     11225  2019-04-29 11:33:00  end
2     11225  2019-04-29 12:03:00  end
3     11225  2019-04-29 12:39:00  end
4     11225  2019-04-29 13:14:00  end


In [14]:
df_merged = df_starts.merge(df_ends, left_on=df_starts.index, right_on=df_ends.index, suffixes=("_start", "_end"))
df_merged.drop(["key_0", 
                "valid_start_start", 
                "valid_start_end", 
                "trip_start", 
                "trip_end", 
                "b_number_end", 
                "b_boardcomputer_end", 
                "b_pedelec_battery_end", 
                "b_lock_types_end", 
                "b_bike_type_end", 
                "b_electric_lock_end"], axis=1, inplace=True)
df_merged.head()

,p_bike_racks_start,p_spot_start,p_booked_bikes_start,p_place_type_start,datetime_start,b_number_start,p_uid_start,p_bikes_start,p_lat_start,b_electric_lock_start,...,p_bikes_end,p_lat_end,p_name_end,p_free_racks_end,p_address_end,p_number_end,p_lng_end,p_maintenance_end,p_terminal_type_end,p_bike_types_end
0,0,True,0,0,2019-04-29 06:31:00,11225,264595,1,51.058771,True,...,1,51.058771,Gutenbergstraße,0,NaN,4389.0,13.769281,False,NaN,"{""15"": 1}"
1,0,False,0,12,2019-04-29 11:27:00,11225,15109859,1,50.806704,True,...,1,50.813655,BIKE 11225,0,NaN,0.0,8.770495,False,NaN,"{""15"": 1}"
2,0,False,0,12,2019-04-29 11:58:00,11225,15117356,1,50.813655,True,...,1,50.808976,BIKE 11225,0,NaN,0.0,8.772863,False,NaN,"{""15"": 1}"
3,0,False,0,12,2019-04-29 12:22:00,11225,15118389,1,50.808976,True,...,1,50.822621,BIKE 11225,0,NaN,0.0,8.774550,False,NaN,"{""15"": 1}"
4,0,False,0,12,2019-04-29 13:00:00,11225,15119670,1,50.822621,True,...,1,50.813241,BIKE 11225,0,NaN,0.0,8.763238,False,NaN,"{""15"": 1}"


## Merge Trips with Weather data

In [15]:
df_dwd=pd.read_csv(os.path.join(path_output_weather, "DWD.csv"))
df_dwd['MESS_DATUM'] = pd.to_datetime(df_dwd['MESS_DATUM'].astype(str), format="%Y-%m-%d %H:%M:%S", errors='coerce')

df_merged["datetime_start"] = pd.to_datetime(df_merged["datetime_start"].astype(str), format="%Y-%m-%d %H:%M:%S", errors="coerce")

In [16]:
df_full = pd.merge_asof(df_merged.sort_values('datetime_start'), 
                        df_dwd, left_on='datetime_start', 
                        right_on='MESS_DATUM', 
                        tolerance=pd.Timedelta('30 min'),
                        allow_exact_matches=True, direction='nearest')

In [17]:
df_full.head(5)

,p_bike_racks_start,p_spot_start,p_booked_bikes_start,p_place_type_start,datetime_start,b_number_start,p_uid_start,p_bikes_start,p_lat_start,b_electric_lock_start,...,p_terminal_type_end,p_bike_types_end,Unnamed: 0,MESS_DATUM,air_deg,air_hum,rain_mm,rain_yn,sun_hour,wind_ms
0,0,False,0,12,2019-01-20 00:00:00,93771,12095573,1,51.071262,False,...,NaN,"{""15"": 1}",1752,2019-01-20,-3.3,75.3,0.0,0.0,0.0,4.3
1,0,True,0,0,2019-01-20 00:05:00,93576,10299640,5,51.038210,False,...,NaN,"{""15"": 1}",1752,2019-01-20,-3.3,75.3,0.0,0.0,0.0,4.3
2,0,True,0,0,2019-01-20 00:07:00,93440,10299584,1,51.042570,True,...,free,"{""undefined"": 1, ""15"": 2}",1752,2019-01-20,-3.3,75.3,0.0,0.0,0.0,4.3
3,0,True,0,0,2019-01-20 00:07:00,93585,264575,5,51.071740,False,...,NaN,"{""undefined"": 3, ""15"": 2}",1752,2019-01-20,-3.3,75.3,0.0,0.0,0.0,4.3
4,0,False,0,12,2019-01-20 00:07:00,93322,12098234,1,51.041798,False,...,NaN,"{""undefined"": 1}",1752,2019-01-20,-3.3,75.3,0.0,0.0,0.0,4.3


In [19]:
df_full.drop("Unnamed: 0", inplace=True, axis=1)
df_full.to_csv(os.path.join(path_output, "Trips.csv"))

In [22]:
df_full.head()

,p_bike_racks_start,p_spot_start,p_booked_bikes_start,p_place_type_start,datetime_start,b_number_start,p_uid_start,p_bikes_start,p_lat_start,b_electric_lock_start,...,p_maintenance_end,p_terminal_type_end,p_bike_types_end,MESS_DATUM,air_deg,air_hum,rain_mm,rain_yn,sun_hour,wind_ms
0,0,False,0,12,2019-01-20 00:00:00,93771,12095573,1,51.071262,False,...,False,NaN,"{""15"": 1}",2019-01-20,-3.3,75.3,0.0,0.0,0.0,4.3
1,0,True,0,0,2019-01-20 00:05:00,93576,10299640,5,51.038210,False,...,False,NaN,"{""15"": 1}",2019-01-20,-3.3,75.3,0.0,0.0,0.0,4.3
2,0,True,0,0,2019-01-20 00:07:00,93440,10299584,1,51.042570,True,...,False,free,"{""undefined"": 1, ""15"": 2}",2019-01-20,-3.3,75.3,0.0,0.0,0.0,4.3
3,0,True,0,0,2019-01-20 00:07:00,93585,264575,5,51.071740,False,...,False,NaN,"{""undefined"": 3, ""15"": 2}",2019-01-20,-3.3,75.3,0.0,0.0,0.0,4.3
4,0,False,0,12,2019-01-20 00:07:00,93322,12098234,1,51.041798,False,...,False,NaN,"{""undefined"": 1}",2019-01-20,-3.3,75.3,0.0,0.0,0.0,4.3
